# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path='./Resources/cities_weather_data.csv'
cities_df=pd.read_csv(path)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Navoiy,40.0844,65.3792,26.60,86,0,6.91,UZ,2021-01-27 19:12:33
1,Kaitangata,-46.2817,169.8464,54.00,70,91,8.01,NZ,2021-01-27 19:12:33
2,Saint-Leu,-21.1500,55.2833,78.01,60,2,5.17,RE,2021-01-27 19:12:33
3,Bredasdorp,-34.5322,20.0403,48.20,81,3,2.35,ZA,2021-01-27 19:12:34
4,Ushuaia,-54.8000,-68.3000,39.20,81,75,10.36,AR,2021-01-27 19:08:43


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#print(gkey)

In [4]:
gmaps.configure(api_key=gkey)

In [5]:
coordinates=cities_df[['Lat','Lng']]
fig = gmaps.figure()
humidity=cities_df["Humidity"]

In [6]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_city_df=cities_df.loc[(cities_df['Max Temp']<80) &
                         (cities_df['Max Temp']>70) &
                         (cities_df['Wind Speed']<10) &
                          (cities_df['Cloudiness']<10),:].dropna()
new_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Saint-Leu,-21.1500,55.2833,78.01,60,2,5.17,RE,2021-01-27 19:12:33
156,Vila Velha,-20.3297,-40.2925,77.00,78,0,5.75,BR,2021-01-27 19:13:08
199,Antalaha,-14.9003,50.2788,75.34,85,0,3.29,MG,2021-01-27 19:11:21
205,Birao,10.2849,22.7882,70.54,22,7,5.93,CF,2021-01-27 19:13:18
216,Changanācheri,9.4667,76.5500,73.47,66,4,2.57,IN,2021-01-27 19:13:20
238,Beloha,-25.1667,45.0500,77.05,76,0,9.10,MG,2021-01-27 19:13:25
249,Esim,4.8699,-2.2405,77.20,90,0,5.75,GH,2021-01-27 19:12:10
333,Tezu,27.9167,96.1667,70.99,20,0,1.34,IN,2021-01-27 19:13:45
371,Sittwe,20.1500,92.9000,70.25,44,0,4.68,MM,2021-01-27 19:13:53
380,Narasaraopet,16.2500,80.0667,73.13,79,2,3.22,IN,2021-01-27 19:13:56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=new_city_df[['City','Country','Lat','Lng']].copy()
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Saint-Leu,RE,-21.1500,55.2833,
156,Vila Velha,BR,-20.3297,-40.2925,
199,Antalaha,MG,-14.9003,50.2788,
205,Birao,CF,10.2849,22.7882,
216,Changanācheri,IN,9.4667,76.5500,
238,Beloha,MG,-25.1667,45.0500,
249,Esim,GH,4.8699,-2.2405,
333,Tezu,IN,27.9167,96.1667,
371,Sittwe,MM,20.1500,92.9000,
380,Narasaraopet,IN,16.2500,80.0667,


In [9]:
params={
    "radius":5000,
    "types":"Lodging",
    "key":gkey
}
hotel_name=[]
for index,rows in hotel_df.iterrows():
    lat=rows['Lat']
    lng=rows['Lng']
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address=requests.get(base_url,params=params).json()
    hotel_name.append(name_address['results'][0]['name'])
hotel_df['Hotel Name']=hotel_name
hotel_df
    

,City,Country,Lat,Lng,Hotel Name
2,Saint-Leu,RE,-21.1500,55.2833,"Saint-Leu, Réunion"
156,Vila Velha,BR,-20.3297,-40.2925,Vila Velha
199,Antalaha,MG,-14.9003,50.2788,Antalaha
205,Birao,CF,10.2849,22.7882,Birao
216,Changanācheri,IN,9.4667,76.5500,Changanassery
238,Beloha,MG,-25.1667,45.0500,Beloha
249,Esim,GH,4.8699,-2.2405,Axim
333,Tezu,IN,27.9167,96.1667,Tezu
371,Sittwe,MM,20.1500,92.9000,Sittwe
380,Narasaraopet,IN,16.2500,80.0667,Narasaraopeta


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))